In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import copy
import matplotlib.pyplot as plt

from utils import MaskedForward

In [2]:
# Configuración
batch_size = 64
threshold_accuracy = 0.60
sparsity_percentage = 0.1  # Porcentaje de pesos a conservar (más altos)
max_epochs_before_restart = 70
min_accuracy_before_restart = 0.50  # 50% accuracy mínima
early_restart_epoch = 30
early_restart_accuracy = 0.40  # 40% accuracy en época 30
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Dataset Animals10
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

data_dir = "../data/animals10"

train_dataset = datasets.ImageFolder(root=os.path.join(data_dir, "train"), transform=transform)
test_dataset = datasets.ImageFolder(root=os.path.join(data_dir, "test"), transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [4]:
net = models.alexnet(pretrained=False)
net.classifier[6] = nn.Linear(net.classifier[6].in_features, 10)

mask = models.alexnet(pretrained=False)
mask.classifier[6] = nn.Linear(mask.classifier[6].in_features, 10)

net.to(device)
mask.to(device)

C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [5]:
# Congelar los parámetros de net (no se entrenarán)
for param in net.parameters():
    param.requires_grad = False

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mask.parameters(), lr=0.001)
epoch = 0
saved = False
losses = []
accuracies = []
restart_count = 0

while True:
    # Reinicializar si es necesario (verificación en época 70)
    if epoch >= max_epochs_before_restart:
        if len(accuracies) == 0 or max(accuracies) < min_accuracy_before_restart * 100:
            restart_count += 1
            print(f"\n!!! RESTART {restart_count} !!!")
            print(f"No se alcanzó {min_accuracy_before_restart*100}% de accuracy en {max_epochs_before_restart} épocas.")
            print("Reinicializando net y máscara, y reiniciando el entrenamiento...")
            
            # Reinicializar net (nueva inicialización aleatoria)
            net = models.alexnet(pretrained=False)
            net.classifier[6] = nn.Linear(net.classifier[6].in_features, 10)
            net.to(device)
            
            # Congelar los parámetros de net
            for param in net.parameters():
                param.requires_grad = False
            
            # Reinicializar la máscara
            mask = models.alexnet(pretrained=False)
            mask.classifier[6] = nn.Linear(mask.classifier[6].in_features, 10)
            mask.to(device)
            
            # Reinicializar el optimizador
            optimizer = optim.Adam(mask.parameters(), lr=0.001)
            
            # Reinicializar métricas
            epoch = 0
            losses = []
            accuracies = []
            saved = False
    
    # Verificación de reinicio temprano en época 30
    if epoch == early_restart_epoch:
        if len(accuracies) == 0 or max(accuracies) < early_restart_accuracy * 100:
            restart_count += 1
            print(f"\n!!! EARLY RESTART {restart_count} !!!")
            print(f"No se alcanzó {early_restart_accuracy*100}% de accuracy en época {early_restart_epoch}.")
            print("Reinicializando net y máscara, y reiniciando el entrenamiento...")
            
            # Reinicializar net (nueva inicialización aleatoria)
            net = models.alexnet(pretrained=False)
            net.classifier[6] = nn.Linear(net.classifier[6].in_features, 10)
            net.to(device)
            
            # Congelar los parámetros de net
            for param in net.parameters():
                param.requires_grad = False
            
            # Reinicializar la máscara
            mask = models.alexnet(pretrained=False)
            mask.classifier[6] = nn.Linear(mask.classifier[6].in_features, 10)
            mask.to(device)
            
            # Reinicializar el optimizador
            optimizer = optim.Adam(mask.parameters(), lr=0.001)
            
            # Reinicializar métricas
            epoch = 0
            losses = []
            accuracies = []
            saved = False
    
    masked_model = MaskedForward(net, mask, sparsity_percentage).to(device)
    # Guardar copia de los pesos de net para verificar que no cambian
    # net_before = [p.clone().detach() for p in net.parameters()]
    masked_model.train()
    running_loss = 0.0

    epoch += 1

    print(f"\n===== Epoch {epoch} (Restart {restart_count}) =====")

        # Guardar copia de los pesos de mask
    # mask_before = [p.clone().detach() for p in mask.parameters()]

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
    
        optimizer.zero_grad()
        outputs = masked_model(images)
        loss = criterion(outputs, labels)
        loss.backward()

        # # Verificar gradientes
        # for name, param in mask.named_parameters():
        #     if param.grad is not None:
        #         print(f"[Grad] {name}: grad norm = {param.grad.norm().item():.6f}")
        #     else:
        #         print(f"[Grad] {name}: NO grad")
        optimizer.step()

        running_loss += loss.item()
        # if batch_idx % 10 == 0:
        #     print(f"Batch {batch_idx}, Loss: {loss.item():.4f}")

    avg_loss = running_loss / len(train_loader)
    losses.append(avg_loss)

    # Evaluación
    masked_model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = masked_model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100.0 * correct / total
    accuracies.append(accuracy)

    #     # Verificar cambios en los parámetros de mask
    # for (before, after), name in zip(zip(mask_before, mask.parameters()), mask.state_dict().keys()):
    #     delta = (after - before).abs().sum().item()
    #     print(f"[Update] {name}: Δ = {delta:.6f}")

    # # Verificar que net no ha sido modificado
    # for (before, after), name in zip(zip(net_before, net.parameters()), net.state_dict().keys()):
    #     delta = (after - before).abs().sum().item()
    #     print(f"[Check Net] {name}: Δ = {delta:.6f}")

    print(f"Epoch {epoch}: Avg Loss = {avg_loss:.4f}, Test Accuracy = {accuracy:.2f}%")

    if accuracy >= threshold_accuracy * 100:
        print("Reached target accuracy. Stopping training.")
        if not saved:
            # Guardar net y mask
            torch.save(net.state_dict(), "net.pth")
            torch.save(mask.state_dict(), "mask.pth")
            print(f"Models saved)")
            saved = True
        else:
            print("Model already saved, skipping save.")
        break


===== Epoch 1 (Restart 0) =====


KeyboardInterrupt: 

In [ ]:
# Gráficos de pérdida y precisión en una sola figura
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Gráfico de pérdida
ax1.plot(range(1, len(losses)+1), losses, label='Loss', color='red')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss per Epoch')
ax1.legend()
ax1.grid(True)

# Gráfico de precisión
ax2.plot(range(1, len(accuracies)+1), accuracies, label='Accuracy', color='green')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Test Accuracy per Epoch')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
print(losses)
print(len(losses))
print(accuracies)
print(len(accuracies))
print(epoch)